In [ ]:
import numpy as np
import pandas as pd
import datetime
from numpy.linalg import inv
from pandas_datareader import data as pdr
import yfinance as yfin
#import matplotlib.pyplot as plt
#from scipy.optimize import minimize
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
start = datetime.datetime(2010,1,1)
end = datetime.datetime.now()


In [ ]:
def data_returns(data): #generating daily avg assets return 
    returns = (data/data.shift(1))-1
    return returns

In [ ]:
def portfolio_covariance(data): #generating yearly avg portfolio risk
    yearly_port_cov = data_returns(data).cov()*252
    return yearly_port_cov
    

#### Markowitz Portfolio Theory Function

In [ ]:
# 1000 combination with function (Simple return)

def combination(Ticker):

    #load data and create random weights dataframe
    data = pd.DataFrame()
    data = pdr.get_data_yahoo(Ticker,start,end)['Adj Close']
    
    weights_array = pd.DataFrame(columns=[Ticker])

    #empty array for Risk and Return Columns
    port_return = []
    port_risk = []
  
    #generating 1000 random combination for the portfolio
    for w in range (1000):
        #get random weights for each Ticker
        weights = np.random.random(len(Ticker)) 

        #weights return array
        weights = weights/np.sum(weights) 
        
        #insert weights (array) to a dataframe called each_weights
        weights_array.loc[len(weights_array)] = weights.tolist()  
        
        #calculate avg annual return and store it to array port_return
        port_return.append(np.sum(weights * (data_returns(data).mean()*252)))
        
        #calculate avg annual std dev/risk and store it to array port_risk
        port_risk.append(np.sqrt(np.dot(weights.T, np.dot(portfolio_covariance(data), weights)))) 


    #create dataframe called portfolios to store risk and return
    portfolios = pd.DataFrame({'Return':port_return, 'Risk': port_risk}) 
    for i in range(len(Ticker)):   
        weights_array.columns.values[i] = Ticker[i]
        
    result = pd.concat([weights_array, portfolios], axis=1) #concat dataframe each_weights and portfolios
    
    #return port_return, port_risk
    return  result

In [ ]:
#Calculation Minimum Variance Portfolio with Lagrange multiplier and Cramer's Rule based on Modul Portfolio Selection from Prof. Dr. Andreas Görg
def Min_Varianz(Ticker):
    
    data = pd.DataFrame()
    data = pdr.get_data_yahoo(Ticker,start,end)['Adj Close']

    #Variable
    Mu = (data_returns(data).mean()*252).to_numpy()
    Sigma = (portfolio_covariance(data)).to_numpy()
    One = [1 for x in range(len(Mu))]
    One_arr = np.array(One)
    Sigma_inverse = inv(Sigma)

    a = np.dot(np.dot(Mu.T,Sigma_inverse),Mu)
    b = np.dot(np.dot(Mu.T,Sigma_inverse),One_arr)
    c = np.dot(np.dot(One_arr,Sigma_inverse),One_arr)


    Return_x = b/c
    Risk_x = 1/np.sqrt(c)

    weights = (np.dot((Return_x*c-b)*Sigma_inverse,Mu)+(np.dot((a-Return_x*b)*Sigma_inverse,One_arr)))/(a*c-b**2)
    
    Min_Port = np.array([Return_x,Risk_x])

    Data = np.concatenate((weights,Min_Port))

    Columns = Ticker
    Columns.append('Return')
    Columns.append('Risk')

    Min_Varianz_Data = pd.DataFrame(columns=Columns)
    Min_Varianz_Data.loc[len(Min_Varianz_Data)] = Data    


    return Min_Varianz_Data

#### Calling Function

In [ ]:
xxx = ['AAPL','TSLA','COKE']

In [ ]:
my_combination = combination(xxx)

In [ ]:
my_combination['Sharpe Ratio'] = (my_combination['Return']-0.0304)/my_combination['Risk']
my_combination

In [ ]:
Sharpe_max = my_combination[my_combination['Sharpe Ratio']== my_combination['Sharpe Ratio'].max()]
Sharpe_max

In [ ]:
Min_Var_Portfolio = Min_Varianz(xxx)
Min_Var_Portfolio

In [ ]:
Min_Var_Portfolio['Sharpe Ratio'] = (Min_Var_Portfolio['Return']-0.0304)/Min_Var_Portfolio['Risk']
Min_Var_Portfolio


In [ ]:
#More Simple Version of Minimum Variance

# Find index of row with minimum risk
min_risk_idx = my_combination['Risk'].idxmin()

# Subset DataFrame to only include rows with minimum risk
min_risk_df = my_combination[my_combination['Risk'] == my_combination.loc[min_risk_idx, 'Risk']]

# Find index of row with maximum return among rows with minimum risk
optimal_return_idx = min_risk_df['Return'].idxmax()

# Subset DataFrame to only include row with both minimum risk and maximum return
optimal_portfolio = min_risk_df[min_risk_df['Return']==min_risk_df.loc[optimal_return_idx,'Return']]

In [148]:
#reformat data
def reformat_Data(Data):
     
     Data_1 = Data
     Data_1.iloc[:,:-3] = (Data_1.iloc[:,:-3]*100).round(2).astype(str) + "%"
     Data_1.iloc[:,-3:] = (Data_1.iloc[:,-3:]*100).round(2)
     
     return Data_1


In [149]:

Scatter_Data = reformat_Data(my_combination)
Min_Varianz_Data = reformat_Data(Min_Var_Portfolio)
Sharpe_max_Data = reformat_Data(Sharpe_max)
Optimal_Portfolio_Data = reformat_Data(optimal_portfolio)

c:\Users\jauha\anaconda3\lib\site-packages\pandas\core\indexing.py:1851: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [151]:
sorted = my_combination.sort_values(by=['Risk'])
test = sorted.groupby(np.arange(len(sorted))//8).max()
test

,AAPL,TSLA,COKE,Return,Risk,Sharpe Ratio
0,62.05%,43.15%,4.83%,28.07,24.45,102.49
1,63.84%,44.53%,8.0%,28.83,24.52,105.32
2,64.81%,45.99%,9.75%,29.26,24.59,106.64
3,62.71%,46.89%,9.95%,29.41,24.65,107.01
4,69.66%,49.31%,7.1%,29.62,24.73,107.60
...,...,...,...,...,...,...
120,9.0%,9.64%,72.85%,46.95,44.56,99.89
121,6.84%,6.48%,75.62%,47.71,45.71,98.17
122,8.72%,9.52%,77.71%,48.38,46.80,97.53
123,8.01%,3.86%,85.71%,50.28,50.32,96.37


In [153]:
sorted.groupby(np.arange(len(sorted))//8)['Sharpe Ratio'].agg(['max'])

,max
0,102.49
1,105.32
2,106.64
3,107.01
4,107.60
...,...
120,99.89
121,98.17
122,97.53
123,96.37


#### Visualization

In [155]:
#Visualizing the 1000 combination data
fig1= px.scatter(data_frame=Scatter_Data,
               x='Risk',y='Return',hover_data=Scatter_Data.columns, color='Sharpe Ratio',
               )

#Visualizing a portfolio with lowest risk
fig2 = px.scatter(data_frame=Min_Varianz_Data, x = 'Risk', y = 'Return',hover_data=Min_Varianz_Data.columns,labels=dict(Risk="Risk (%)", Return="Return (%)"))
#Visualizing a portfolio with highest sharpe ratio
fig3 = px.scatter(data_frame=Sharpe_max_Data, x = 'Risk', y = 'Return',hover_data=Sharpe_max_Data.columns,labels=dict(Risk="Risk (%)", Return="Return (%)"))

fig4 =px.scatter(data_frame=optimal_portfolio,x = 'Risk', y = 'Return',hover_data=optimal_portfolio.columns,labels=dict(Risk="Risk (%)", Return="Return (%)"))

#adding marker to fig2 and fig3
fig2.update_traces(marker=dict(size=15, color="Red",symbol="x"))
fig3.update_traces(marker=dict(size=15, color="Green",symbol="x"))
fig4.update_traces(marker=dict(size=15, color="Blue",symbol="x"))


#here i'm trying to draw a efficient frontier line from the visualization but still not getting the result i wanted
fig5= px.line(data_frame=test,
               x='Risk',y='Return'
               )


fig = go.Figure(data = fig1.data + fig2.data + fig3.data+fig4.data+fig5.data)
fig.update_layout(
    title="Portfolio Combination",
    xaxis_title="Risk (%)",
    yaxis_title="Return (%)",
    font=dict(
        family="Bahnschrift",
        size=18,
        color="RebeccaPurple"
    )
)
fig.show()